# Monads for Normal Programmers

http://blog.jorgenschaefer.de/2013/01/monads-for-normal-programmers.html

## Example 1: jQuery

jQuery is a JavaScript library for website manipulation. One of its main features is a monad. No one calls it that, though.

```javascript
$("#element").find("ul").parent().hide()

```

This expression searches for an element with the id #element, finds unsorted lists (ul) among the childs of that element, finds the parents of those childs, and hides them.

Each of those expressions return an object representing a “set of matched elements” (I’m not even sure if jQuery defines a single name for that concept), and the same kind of methods can be applied to all such objects. This means you can chain such calls as shown above.

Branching looks very similar:

```javascript
var lists = $("#element").find("ul");
var parents = lists.parent();
var list_items = lists.children("li");

```

In this example, we first split up the original query. The variable lists now contains an object representing the set of unsorted list tags under the #element tag. We can now re-use this set. First, we find the parent elements of those unsorted lists. Then we find their list items.

Because every one of those calls returns the same kind of object which provides the same methods which all return such objects, we were able to both simply chain multiple calls to each other, but also to sort of cache an intermediate result and split execution from that point.

Both are very powerful programming methods.


## Example 2: Django Query Sets



With its queryset objects, Django provides a monad for database abstractions. A query set represents a partially-constructed SQL statement. By using method application, you can successively build the expression.

https://docs.djangoproject.com/en/2.0/topics/db/queries/


> BTW: Just look at all the cruft you need to do - just to use the Django ORM models
> https://stackoverflow.com/questions/937742/use-django-orm-as-standalone

```python
Book.objects.filter(author="Bernard Cornwell")
            .exclude(series="The Sharpe stories")
```

Chaining has an extremely useful application in combination with conditions:

```python
books = Book.objects.all()
if author is not None:
   books = books.filter(author=author)
if year is not None:
   books = books.filter(year=year)
```

In this case, the links of the chain are added conditionally depending on specified options, for example to a function. This is a very powerful implication of the ability to chain monadic types.

Branching works, too:

```python
cornwell = Book.objects.filter(author="Bernard Cornwell")
older = cornwell.filter(published__lt="2000")
new = cornwell.filter(published__gt="2010")
```

One of the main drawbacks of SQL is that it’s so difficult to compose statements. Even just adding a simple added WHERE clause is complicated enough, successively adding queries to other tables and other more complex problems is almost impossible. Abstracting them using monadic types allows for very strong compositability.



## Type Theory
So far, you have kindly just believed me that those examples actually are monads. Let me fulfill my promise from earlier and show that they actually are. And not only that, but also that all monads are like that. That is, that the following definition is equivalent to the type theoretical definition of monads:

On short...

> A Monad is an object whose methods return monads.


## Addendum: Monad Laws

The first two monad laws are about identy, namely that bind and unit form left and right identities.

```python
bind(unit(x), f) == f(x)
```

The second identity law is as follows.

```python
bind(m, unit) == m
```

The third law is the most complex.

```python
(m >>= f) >>= g ≡ m >>= ( \x -> (f x >>= g) )
```

Or, to write it in our syntax:

```python
m().f().g() ≡  m().fun()
fun(x) ::= f(x).g()
```

m().fun() is fun(m()). Expanding that function gives us f(m()).g(), which in turn gives us g(f(m())). Which actually is the same as m().f().g().


# Monads for Normal Programmers (part 2)



## Finding Monads

Let’s start with a concrete problem. What I would like is a class which encapsulates chained mathematical operations.

```python
>>> MathOp(5)
<MathOp 5>
>>> MathOp(5).mul(2)
<MathOp 10>
>>> MathOp(5).mul(2).add(17)
<MathOp 27>
>>> MathOp(5).mul(2).add(17).sub(4)
<MathOp 23>
```

So far, so simple, but let’s add a twist. If there is a div(0) anywhere in the chain, we don’t want to raise an exception, but to have the whole chain return a NaN.

```python
>>> MathOp(5).div(0)
<MathOp NaN>
>>> MathOp(5).div(0).mul(2)
<MathOp NaN>
>>> MathOp(5).div(0).mul(2).add(17)
<MathOp NaN>
>>> MathOp(5).div(0).mul(2).add(17).sub(4)
<MathOp NaN>
```

## Step 1: Trivial Implementation

The first implementation of this is straightforward and simple.

In [97]:
class MathOp(object):
    def __init__(self, value=None, is_nan=False):
        self.value = value
        self.is_nan = is_nan

    def __repr__(self):
        if self.is_nan:
            return "<MathOp NaN>"
        else:
            return "<MathOp {}>".format(self.value)

    def div(self, denum):
        if self.is_nan:
            return self
        elif denum == 0:
            return MathOp(is_nan=True)
        else:
            return MathOp(self.value / denum)

    def mul(self, multiplicand):
        if self.is_nan:
            return self
        else:
            return MathOp(self.value * multiplicand)

    def add(self, multiplicand):
        if self.is_nan:
            return self
        else:
            return MathOp(self.value + multiplicand)

    def sub(self, multiplicand):
        if self.is_nan:
            return self
        else:
            return MathOp(self.value - multiplicand)

## Step 2: Bind

When you look at that code, you will notice that all the methods implementing math operations share the same test for is_nan. This repetitive code could be abstracted out into a common pattern, maybe using a decorator.

In [98]:
from functools import wraps 

In [99]:
def mathop(method):
    @wraps(method)
    def math_method(self, *args, **kwargs):
        if self.is_nan:
            return self
        else:
            return method(self, *args, **kwargs)
    return math_method

This lets us write the class a bit more succinctly:

In [100]:
class MathOp(object):
    def __init__(self, value=None, is_nan=False):
        self.value = value
        self.is_nan = is_nan

    def __repr__(self):
        if self.is_nan:
            return "<MathOp NaN>"
        else:
            return "<MathOp {}>".format(self.value)

    @mathop
    def div(self, denum):
        if denum == 0:
            return MathOp(is_nan=True)
        else:
            return MathOp(self.value / denum)

    @mathop
    def mul(self, multiplicand):
        return MathOp(self.value * multiplicand)

    @mathop
    def add(self, addend):
        return MathOp(self.value + addend)

    @mathop
    def sub(self, subtrahend):
        return MathOp(self.value - subtrahend)

This common pattern is called bind in monad theory. If you remember my last article, I noted that bind is method application. This here now is the implementation of that idea. Bind influences how methods are applied.

To make the idea more general, we could define a bind method in our class and make our decorator call that instead.

In [101]:
def bound(method):
    @wraps(method)
    def bound_method(self, *args, **kwargs):
        return self.bind(method, args, kwargs)
    return bound_method

By defining the bind magic in the class itself, we can re-use this decorator in different subclasses.

In [102]:
class MathOp(object):
    def __init__(self, value=None, is_nan=False):
        self.value = value
        self.is_nan = is_nan

    def __repr__(self):
        if self.is_nan:
            return "<MathOp NaN>"
        else:
            return "<MathOp {}>".format(self.value)

    def __eq__(self, mathop):
        return (self.is_nan == mathop.is_nan) & (self.value == mathop.value)

    def bind(self, method, args, kwargs):
        if self.is_nan:
            return self
        else:
            return method(self, *args, **kwargs)

    @bound
    def div(self, denum):
        if denum == 0:
            return MathOp(is_nan=True)
        else:
            return MathOp(self.value / denum)

    @bound
    def mul(self, multiplicand):
        return MathOp(self.value * multiplicand)

    @bound
    def add(self, addend):
        return MathOp(self.value + addend)

    @bound
    def sub(self, subtrahend):
        return MathOp(self.value - subtrahend)

In [103]:
print(MathOp(5) == MathOp(5))
print(MathOp(5).mul(2) == MathOp(10))
print(MathOp(5).mul(2).add(17) == MathOp(27))
print(MathOp(5).mul(2).add(17).sub(4) == MathOp(23))

True
True
True
True


In [104]:
print(MathOp(5).div(0) == MathOp(is_nan=True))
print(MathOp(5).div(0).mul(2) == MathOp(is_nan=True))
print(MathOp(5).div(0).mul(2).add(17) == MathOp(is_nan=True))
print(MathOp(5).div(0).mul(2).add(17).sub(4) == MathOp(is_nan=True))


True
True
True
True


This is the main part of the monad design pattern: We can abstract away some specifics related to method application by using a decorator that calls the bind method for us first.

Admittedly, the name bind is terrible, but it’s what the monad theory uses. Use something more intuitive when you actually use this pattern.

Note: It is possible to use metaclasses to make all methods of a class use bind by default. This then requires a decorator for methods that should not be passed through bind. It also adds a lot of magic which I do not think helps understanding at all.

## Step 3: Maybe Monad

You can probably see that what we implemented here is again an example of a generic pattern: A chain of operations continues until a “bad” value happens, then all following operations just pass on the bad value. This is useful for mathematical operations, but it could be useful for other things, too, if throwing an exception is not a sensible solution. Some languages even implement exceptions using this pattern.

This generic pattern is called the maybe monad.


In [105]:
class MaybeMonad(object):
    is_nothing = False

    def bind(self, method, args, kwargs):
        if self.is_nothing:
            return self
        else:
            return method(self, *args, **kwargs)

We can subclass this to implement our math operation. As the is_nothing field is now available on the class level, we can simply define our NaN value as a subclass of MathOp. This would be a good place for a singleton, but that’s a bit outside of the topic of this article.

In [106]:
class MathOp(MaybeMonad):
    is_nothing = False

    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return "<MathOp {}>".format(self.value)

    def __eq__(self, mathop):
        return (self.is_nothing == mathop.is_nothing) & (self.value == mathop.value)

    @bound
    def div(self, denum):
        if denum == 0:
            return MathOpNaN()
        else:
            return MathOp(self.value / denum)

    @bound
    def mul(self, multiplicand):
        return MathOp(self.value * multiplicand)

    @bound
    def add(self, addend):
        return MathOp(self.value + addend)

    @bound
    def sub(self, subtrahend):
        return MathOp(self.value - subtrahend)

class MathOpNaN(MathOp):
    is_nothing = True

    def __init__(self):
        super(MathOpNaN, self).__init__(None)

    def __repr__(self):
        return "<MathOp NaN>"

In [107]:
print(MathOp(5) == MathOp(5))
print(MathOp(5).mul(2) == MathOp(10))
print(MathOp(5).mul(2).add(17) == MathOp(27))
print(MathOp(5).mul(2).add(17).sub(4) == MathOp(23))

True
True
True
True


In [129]:
print(MathOp(5).div(0) == MathOpNaN())
print(MathOp(5).div(0).mul(2) == MathOpNaN())
print(MathOp(5).div(0).mul(2).add(17) == MathOpNaN())
print(MathOp(5).div(0).mul(2).add(17).sub(4) == MathOpNaN())

True
True
True
True


## Step 4: Monads!

So our MathOp class is an instance of a generic design pattern, called the Maybe Monad. The same design pattern can be used for other purposes.

A step further though, the Maybe Monad is itself an instance of an even more generic design pattern. This more generic design pattern is about the ability to chain method calls, and to override what method application does, exactly. This more general design pattern is, as you likely guessed, the Monad.

In [130]:
class Monad(object):
    def bind(self, method, args, kwargs):
        return method(self, *args, **kwargs)

def bound(method):
    @wraps(method)
    def bound_method(self, *args, **kwargs):
        result = self.bind(method, args, kwargs)
        assert(isinstance(result, Monad))
        return result
    return bound_method

This example code for bound enforces that the result of bind actually is a monad itself. That’s not something we need to do in a dynamically typed language like Python, but it highlights one of the theoretical requirements of monads.

We can now use this meta-pattern to define our MaybeMonad.

In [133]:
class MaybeMonad(Monad):
    is_nothing = False

    def bind(self, method, args, kwargs):
        if self.is_nothing:
            return self
        else:
            return method(self, *args, **kwargs)

And in turn, we can use this MaybeMonad pattern to define the MathOp class.

In [136]:
class MathOp(MaybeMonad):
    is_nothing = False

    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return "<MathOp {}>".format(self.value)

    def __eq__(self, mathop):
        return (self.is_nothing == mathop.is_nothing) & (self.value == mathop.value)

    @bound
    def div(self, denum):
        if denum == 0:
            return MathOpNaN()
        else:
            return MathOp(self.value / denum)

    @bound
    def mul(self, multiplicand):
        return MathOp(self.value * multiplicand)

    @bound
    def add(self, addend):
        return MathOp(self.value + addend)

    @bound
    def sub(self, subtrahend):
        return MathOp(self.value - subtrahend)

class MathOpNaN(MathOp):
    is_nothing = True

    def __init__(self):
        super(MathOpNaN, self).__init__(None)

    def __repr__(self):
        return "<MathOp NaN>"

In [137]:
print(MathOp(5) == MathOp(5))
print(MathOp(5).mul(2) == MathOp(10))
print(MathOp(5).mul(2).add(17) == MathOp(27))
print(MathOp(5).mul(2).add(17).sub(4) == MathOp(23))

True
True
True
True


In [138]:
print(MathOp(5).div(0) == MathOpNaN())
print(MathOp(5).div(0).mul(2) == MathOpNaN())
print(MathOp(5).div(0).mul(2).add(17) == MathOpNaN())
print(MathOp(5).div(0).mul(2).add(17).sub(4) == MathOpNaN())

True
True
True
True


## Conclusions

In a sense, we have now created an abstraction tower.

The term monad refers to a very generic pattern about objects whose methods return monadic objects so that method calls can be chained, together with a generic way with which the application of those methods can be influenced.

Using this general pattern, we can define more specific patterns that use the general pattern for more specific purposes. The Maybe Monad is one example of this, but there are many others. These are still patterns, not concrete, useful classes, though.

We then can take this monadic pattern and create a specific class which implements this pattern and finally have something useful.

Monads are meta patterns for specific monadic structures, which are patterns for actually useful classes.

And just like with other design patterns, the important part is not that you create the full abstraction tower in your code, it’s that you use the pattern. A class that implements a singleton is still a singleton, even if it does not inherit from a singleton abstract class. A factory method is still a factory method, even if it does not implement an abstract factory method interface.

This is especially true for dynamically-typed languages, and probably the greatest source of confusion for people coming from the statically typed world. Python objects do not need a rigid type hierarchy and type checks to implement patterns that require careful consideration for the type system in other languages.

The first implementation we used here is still monadic, even though we did not make bind explicit. It’s useful to see that there is a common pattern there, but it’s not necessary to actually make this pattern explicit in your code. If you have your software engineer hat on, your goal is to create programs that solve problems, and this is the most important thing you care about.

On the other hand, if you are wearing your computer scientist hat, your focus is on analyzing the patterns in programs first, and solving real-world problems only second. This is where it becomes important to split up those patterns, to make sure you know exactly what you are talking about and how the different concepts interact.

## State Monad

Let’s add some complexity. The state monad is a design pattern where you use method chaining to create a computation. The resulting computation then can be run against various inputs (initial states). This is a very handy pattern for all sorts of tasks, like creating a pattern matcher from a pattern specification or pre-computing a composable sequence of actions.

The basic idea is that we define methods in our monad instances that add computations. At the end, we call the accumulated function with an initial state. That state then is passed through the accumulated computation.

Our goal is a simple game (courtesy of haskell.org). This game is played in moves. Each move is either an a, a b, or a c. If the game is on, an a will add one point and b will deduce one point. If the game is off, neither one does anything. A c will toggle the game between on and off. The game starts in the off state.

```python
>>> start_state = (False, 0)
>>> Game().move('a').move('b').run(start_state)
(False, 0)
>>> game = Game().move('c').move('a')
>>> game.run(start_state)
(True, 1)
>>> game.move('b').move('c').move('a').run(start_state)
(False, 0)
```

We can start by defining a very skeleton StateMonad class. We want to store a computation (function), and when we’re done building the computation, we want to run that computation on a start state.

In [140]:
class StateMonad(Monad):
    def __init__(self, function=lambda x: x):
        self.function = function

    def run(self, state):
        return self.function(state)

This isn’t too exciting yet. The default computation even is a real no-op, simply returning what we pass in. But we’re still looking for this generic pattern, not so much writing it down right away.

The first attempt below defines a method in move, which represents the computation which we simply accumulate during the chaining process. Once the game is done, we simply run the resulting game by applying the function to the initial state.

In [141]:
class Game(StateMonad):
    def move(self, char):
        def transformer(state):
            on, score = self.function(state)
            if char == 'a' and on:
                return (on, score + 1)
            elif char == 'b' and on:
                return (on, score - 1)
            elif char == 'c':
                return (not on, score)
            else:
                return (on, score)
        return Game(transformer)

Each computation calls the former computation (self.function), and adds something to its result.

The pattern you can see here is the nested method. On the first pass, when the methods are chained, only move is called, passing in the char parameter. On the second pass, when the resulting game function is applied to the initial state, the inner transformer function is called and actually does the state transition. This is a perfectly fine solution for us, and probably all you’ll ever use in Python when you use this kind of pattern, but as we’re looking at formalizing design patterns, we can try and abstract this further.

The following bind implementation captures the pattern we established above, making the code a bit cleaner:

In [142]:
class Game(StateMonad):
    def bind(self, method, args, kwargs):
        def transformer(old_state):
            current_state = self.run(old_state)
            new_state = method(self, current_state,
                               *args, **kwargs)
            return new_state
        return Game(transformer)

    @bound
    def move(self, state, char):
        on, score = state
        if char == 'a' and on:
            return (on, score + 1)
        elif char == 'b' and on:
            return (on, score - 1)
        elif char == 'c':
            return (not on, score)
        else:
            return (on, score)

So far, so good. For a general pattern, though, it would be really nice if our move method could return a whole series of transitions, not just a single new state. But if this method can return a chained game, then we need a state to run that returned value on.

```python
    def bind(self, method, args, kwargs):
        def transformer(old_state):
            current_state = self.run(old_state)
            new_game = method(self, current_state,
                              *args, **kwargs)
            new_state = new_game.run(???)
            return new_state
        return Game(transformer)
```

The ??? there can be solved by having run return two values, a value and a state. The value is passed to the method, and the state to the run method of the response of the game.

```python
    def bind(self, method, args, kwargs):
        def transformer(old_state):
            value, current_state = self.run(old_state)
            new_game = method(self, value, *args, **kwargs)
            new_value, new_state = new_game.run(current_state)
            return new_value, new_state
        return Game(transformer)
```

So a run of a chained game now returns two values. The first value is passed to the next method, while the second value is passed to the composed state transition functions we create within those methods.

This requires some changes. First, we change run so it returns only the first of the tuple returned, because that’s the actual value. Then we define a _run internal method that returns the tuple for internal use (in bind).


In [143]:
def run(self, state):
    return self.get().function(state)[0]

def _run(self, state):
    return self.function(state)